In [1]:
#Librerias
from pgmpy.models import BayesianNetwork;from pgmpy.factors.discrete import TabularCPD;import dash;from dash import dcc ;from dash import html;from dash.dependencies import Input, Output;import plotly.express as px;from pgmpy . inference import VariableElimination
from pgmpy . sampling import BayesianModelSampling;from pgmpy . estimators import MaximumLikelihoodEstimator;
from pgmpy . estimators import BayesianEstimator;import pandas as pd;from pgmpy . inference import VariableElimination
import numpy as np
from pgmpy.factors.discrete import DiscreteFactor
from sklearn.model_selection import train_test_split
from pgmpy.readwrite import BIFReader; from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, confusion_matrix
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score
from pgmpy.estimators import BicScore
from pgmpy.estimators import PC


c:\Users\Santiago Zubieta\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Traer datos de train y test
X_train=pd.read_csv('X_train')
X_train=X_train.iloc[:, 2:]

X_test=pd.read_csv('X_test')
X_test=X_test.iloc[:, 1:]

Y_test=pd.read_csv('Y_test')
Y_test=Y_test.iloc[:, 1:]

Y_train=pd.read_csv('Y_train')
Y_train=Y_train.iloc[:, 2:]

X_sup=pd.read_csv('X_train')
Y_sup=pd.read_csv('Y_train')


datos_Train=pd.merge(X_sup,Y_sup[['id','hd']],on='id',how='inner')
datos_Train=datos_Train.iloc[:,2:]
print(datos_Train.head())

datos_totales=pd.read_csv('datostotales')
datos_totales=datos_totales.iloc[:,1:]



           age  sex   cp       trestbps    chol  fbs  restecg thalach  exang  \
0  AdultoMayor  1.0  3.0  TensionNormal  Normal  0.0      2.0    Alta    0.0   
1       Adulto  1.0  2.0  TensionNormal    Alto  0.0      0.0    Alta    0.0   
2       Adulto  1.0  2.0  TensionNormal    Alto  1.0      0.0  Normal    0.0   
3       Adulto  1.0  3.0  TensionNormal    Alto  0.0      0.0    Alta    0.0   
4       Adulto  1.0  2.0  TensionNormal    Alto  0.0      0.0    Alta    0.0   

  oldpeak  slope   ca  thal  hd  
0    Alto    2.0  0.0   3.0   1  
1  Normal    1.0  0.0   7.0   0  
2  Normal    1.0  0.0   7.0   0  
3  Normal    1.0  0.0   3.0   0  
4  Normal    1.0  0.0   3.0   0  


In [4]:
#Estimando por medio de restricciones
est=PC(data=datos_Train)
modeloRest=est.estimate(variant='stable',max_cond_vars=5)


Working for n conditional variables: 1:  20%|██        | 1/5 [00:00<00:01,  2.75it/s]c:\Users\Santiago Zubieta\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
c:\Users\Santiago Zubieta\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
c:\Users\Santiago Zubieta\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\CITests.py:548: FutureWarning: In a future version of pandas, a lengt

In [5]:
#Aca esta el modelo
print(modeloRest)
print(modeloRest.nodes())
print(modeloRest.edges())

DAG with 9 nodes and 7 edges
['thalach', 'slope', 'ca', 'hd', 'cp', 'thal', 'oldpeak', 'exang', 'sex']
[('thalach', 'slope'), ('ca', 'hd'), ('cp', 'hd'), ('thal', 'hd'), ('oldpeak', 'slope'), ('exang', 'cp'), ('sex', 'thal')]


In [6]:
#Creando el modelo
modeloRestBN=BayesianNetwork(modeloRest)
print(modeloRestBN.nodes())
modeloRestBN.fit(data=datos_Train,estimator=MaximumLikelihoodEstimator)


['thalach', 'slope', 'ca', 'hd', 'cp', 'thal', 'oldpeak', 'exang', 'sex']


In [3]:
#escribir el modelo 
from pgmpy.readwrite import BIFWriter
from pgmpy.models import BayesianModel

writer=BIFWriter(modeloRestBN)

writer.write_bif(filename="ModeloRestringido.bif")

NameError: name 'modeloRestBN' is not defined

In [7]:
#Puntaje K2
from pgmpy.readwrite import BIFWriter
from pgmpy.models import BayesianModel

scoring_method=K2Score(data=datos_Train)
esth = HillClimbSearch(data=datos_Train)
modeloK2= esth.estimate(
    scoring_method=scoring_method, max_indegree=5, max_iter=int(1e4), black_list=[('age','sex'),('thal','sex'),('chol', 'sex'), ('fbs', 'sex'), ('oldpeak', 'age')]
)
print(modeloK2)
print(modeloK2.nodes())
print(modeloK2.edges())

  0%|          | 18/10000 [00:01<17:56,  9.27it/s] 

DAG with 14 nodes and 17 edges
['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hd']
[('sex', 'thal'), ('sex', 'hd'), ('cp', 'exang'), ('trestbps', 'oldpeak'), ('trestbps', 'cp'), ('trestbps', 'hd'), ('chol', 'thalach'), ('fbs', 'hd'), ('exang', 'thalach'), ('slope', 'oldpeak'), ('slope', 'thalach'), ('ca', 'hd'), ('thal', 'oldpeak'), ('hd', 'thal'), ('hd', 'cp'), ('hd', 'slope'), ('hd', 'exang')]


In [18]:
#Escribir en BIF el modelo K2
modeloK2BN=BayesianNetwork(modeloK2)
writerK2=BIFWriter(modeloK2BN)

modeloK2BN.fit(data=datos_Train,estimator=MaximumLikelihoodEstimator)

writer=BIFWriter(modeloK2BN)
writer.write_bif(filename="ModeloK2.bif")


In [20]:
#Puntaje BIC

scoring_method=BicScore(data=datos_Train)
esth = HillClimbSearch(data=datos_Train)
modeloBic= esth.estimate(
    scoring_method=scoring_method, max_indegree=5, max_iter=int(1e4),black_list=[('age','sex'),('thal','sex'),('chol', 'sex'), ('hd', 'sex'), ('fbs', 'sex'), ('oldpeak', 'age')]
)
print(modeloBic)
print(modeloBic.nodes())
print(modeloBic.edges())

  0%|          | 11/10000 [00:01<22:11,  7.50it/s] 

DAG with 14 nodes and 10 edges
['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'hd']
[('sex', 'thal'), ('sex', 'hd'), ('thalach', 'slope'), ('exang', 'cp'), ('slope', 'oldpeak'), ('hd', 'thal'), ('hd', 'exang'), ('hd', 'thalach'), ('hd', 'cp'), ('hd', 'ca')]


In [24]:
modeloBicBN=BayesianNetwork(modeloBic)

modeloBicBN.fit(data=datos_Train,estimator=MaximumLikelihoodEstimator)
writerBic=BIFWriter(modeloBicBN)
writer=BIFWriter(modeloBicBN)
writerBic.write_bif(filename="ModeloBic.bif")
